In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cvxpy as cp

In [3]:
sns.set_theme(context="notebook")

## Import data

In [4]:
def get_demand_mask_and_price(df, colName, dt):
    col = df[colName].to_numpy() / dt
    mask = col > 0
    price = col[mask][0]
    return mask, price

In [16]:
##### loads
df = pd.read_csv("site1_load.csv")
dt = 1/4 # in hours
powerLoad, heatLoad = df['Power Load [kWh]'].to_numpy(), df['Heat Load [kWh]'].to_numpy()

##### prices
df = pd.read_csv("power_price_B20.csv")
energyPricePower = df["energyPrice"].to_numpy()
# from per kW to per kWh in 15 minutes
powerDemand = [get_demand_mask_and_price(df, colName, dt) for colName in ["peakDemandSummerPrice",
                                                                          "partialPeakDemandSummerPrice",
                                                                          "demandSummerPrice",
                                                                          "peakDemandWinterPrice",
                                                                          "demandWinterPrice"]]
df = pd.read_csv("gas_price.csv")
energyPriceGas = df["energyPrice"].to_numpy()

##### emissions
df = pd.read_csv("power_grid_emissions.csv")
df.ffill(inplace=True)
emissionsPower = df["MOER version 2.0"].to_numpy()
df = pd.read_csv("gas_emissions.csv")
emissionsGas = df["gasEmissions"].to_numpy()

# clear memory
del df

In [6]:
n = len(powerLoad)

## Baseline

We only have a natural gas furnace to supply heat, and no storage.

In [22]:
##### variables and parameters
# financial
CRF = cp.Parameter(nonneg=True) # capital recovery factor TBD
# gas furnace
gasInputGF = cp.Variable(n, nonneg=True)
capaPriceGF = cp.Parameter(nonneg=True) # $/kW TBD
effGF = cp.Parameter(nonneg=True) # %

##### useful quantities
# gas furnace
heatOutputGF = effGF*gasInputGF
capaGF = cp.max(heatOutputGF / dt)
# consumption
gasConsumption = gasInputGF
powerConsumption = powerLoad + 0 # no heat pump or batteries here
# loads
heatSupply = heatOutputGF
# costs
opexPower = energyPricePower@powerConsumption + np.sum([cp.max(powerConsumption[d[0]])*d[1] for d in powerDemand])
opexGas = energyPriceGas@gasConsumption
opex = opexPower + opexGas
capexGF = capaGF*capaPriceGF
capex = capexGF
# emissions
emissions = emissionsPower@powerConsumption + emissionsGas@gasConsumption

##### constraints
cons = []
# meet load
cons += [heatSupply == heatLoad]                                  

##### objective function
obj = cp.Minimize(opex + CRF*capex)

#### solve problem
CRF.value = 1/20 # % TBD
capaPriceGF.value =  200 # $/kW TBD
effGF.value = 0.85 # %
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.CLARABEL)
print(prob.status)
print('cost = ', np.round(obj.value), "$ per year")
print('emissions = ', np.round(emissions.value), "kgCO2 per year")
# TODO LCOS and LCOE, same for emissions

optimal
cost =  5674230.0 $ per year
emissions =  16043137.0 kgCO2 per year


## With a heat pump

Natural gas furnace and heat pump